##  Chunking - Optimizing Vector Database Data Preparation

- Character/Token Based Chunking
- Recursive Character/Token Based Chunking
- Semantic Chunking
- Cluster Semantic Chunking
- LLM Semantic Chunking